In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import os

import numpy as np
import matplotlib.pyplot as plt

from NormalizingFlows.src.train import train_backward
from NormalizingFlows.src.scores import log_likelihood
from NormalizingFlows.src.utils import update_device, load_best_model, load_checkpoint_model

from NormalizingFlows.src.flows import *
from NormalizingFlows.src.data.density.toydata import ToyDataset

In [3]:
def set_visible_devices(*devices: int) -> None:
    '''Utility to set visible Cuda devices.

    Parameters
    ----------
    devices : List[int]
        Index of cuda devices to make available for use.
    '''
    assert all([d >= 0 for d in devices]), f"Not all devices are CUDA devices!"
    os.environ['CUDA_VISIBLE_DEVICES'] = ",".join([str(i) for i in devices])
    

def set_devices(*devices: int):
    '''Utility to set Cuda device(s).

    Parameters
    ----------
    devices : List[int]
        Index of cuda devices to make available for use.

    Returns
    -------
    torch.device or List[torch.device] of currently available CUDA devices.
    '''
    assert len(devices) > 0, f'Device list is empty, no devices set.'
    if len(devices) == 1:
        if devices[0] >= 0:
            set_visible_devices(devices[0])
            return torch.device(0)
        else:
            return torch.device('cpu')

    else:
        set_visible_devices(*devices)
        return [torch.device(i) for i in range(len(devices))]

In [4]:
device = set_devices(1) #torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device_cpu = torch.device("cpu")

In [5]:
dim_input = 25
dataset = ToyDataset(dim_input=dim_input, samples=10000)

dim_input = dataset.dim_input
num_trans = 6 #Must be even
perm_types = ['identity', 'alternate','random']
dim_hidden = [100,100,100]
flows = {}
flow_forward = False


epochs = 15
batch_size = 32
num_exp = 5

In [6]:
#Linear and mean field

name = 'Mean field'
flows[name] = []
for i in range(num_exp):
    transformations = create_constant_trans(num_trans, dim_input, flow_forward, a_param=F.softplus)
    mean_field = create_flows_with_identity(dim_input, transformations, flow_forward) 
    flows[name] += [mean_field]
    flows[name][-1].name = f'{name} {i}'

name = 'Linear'
flows[name] = []
for i in range(num_exp):
    transformations = create_linear_bias_trans(num_trans, dim_input, flow_forward)
    linear = create_flows_with_identity(dim_input, transformations, flow_forward)
    flows[name] += [linear]
    flows[name][-1].name = f'{name} {i}';

In [7]:
# #Affine flows with two block

# name = 'Affine coupling identity'
# flows[name] = []
# for i in range(num_exp):
#     transformations = create_affine_trans(num_trans, flow_forward, a_param=F.softplus)
#     aff_coup_id = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
#     flows[name] += [aff_coup_id]
#     flows[name][-1].name = f'{name} {i}'

# name = 'Affine coupling alternating'
# flows[name] = []
# for i in range(num_exp):
#     transformations = create_affine_trans(num_trans, flow_forward, a_param=F.softplus)
#     aff_coup_alt = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
#     flows[name] += [aff_coup_alt]
#     flows[name][-1].name = f'{name} {i}'
    
# name = 'Affine coupling random'
# flows[name] = []
# for i in range(num_exp):
#     transformations = create_affine_trans(num_trans, flow_forward, a_param=F.softplus)
#     aff_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
#     flows[name] += [aff_coup_rand]
#     flows[name][-1].name = f'{name} {i}'

In [8]:
# #Affine flows with AR

# transformations = create_affine_trans(num_trans, dim_input, flow_forward)
# aff_ar_id = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
# name = 'Affine AR identity'
# flows.append(aff_ar_id), names.append(name)

# transformations = create_affine_trans(num_trans, dim_input, flow_forward)
# aff_ar_alt = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
# name = 'Affine AR alternate'
# flows.append(aff_ar_alt), names.append(name)

# transformations = create_affine_trans(num_trans, dim_input, flow_forward)
# aff_ar_rand = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
# name = 'Affine AR random'
# flows.append(aff_ar_rand), names.append(name);

In [9]:
# #PiecewiseAffine flows with coupling

# transformations = create_piecewise_trans(num_trans, flow_forward)
# piec_coup_id = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
# name = 'PiecewiseAffine coupling identity'
# flows.append(piec_coup_id), names.append(name)

# transformations = create_piecewise_trans(num_trans, flow_forward)
# piec_coup_alt = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
# name = 'PiecewiseAffine coupling alternate'
# flows.append(piec_coup_alt), names.append(name)

# transformations = create_piecewise_trans(num_trans, flow_forward)
# piec_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
# name = 'PiecewiseAffine coupling random'
# flows.append(piec_coup_rand), names.append(name);

In [10]:
# #PiecewiseAffine flows with AR

# transformations = create_piecewise_trans(num_trans, flow_forward)
# piec_ar_id = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
# name = 'PiecewiseAffine AR identity'
# flows.append(piec_ar_id), names.append(name)

# transformations = create_piecewise_trans(num_trans, flow_forward)
# piec_ar_alt = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
# name = 'PiecewiseAffine AR alternate'
# flows.append(piec_ar_alt), names.append(name)

# transformations = create_piecewise_trans(num_trans, flow_forward)
# piec_ar_rand = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
# name = 'PiecewiseAffine AR random'
# flows.append(piec_ar_rand), names.append(name);

In [11]:
# #PiecewiseAffineAffine with twoblock

# transformations = create_affinepiecewise_trans(num_trans, flow_forward)
# piecaf_coup_id = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
# name = 'PiecewiseAffineAffine coupling identity'
# flows.append(piecaf_coup_id), names.append(name)

# transformations = create_affinepiecewise_trans(num_trans, flow_forward)
# piecaf_coup_alt = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
# name = 'PiecewiseAffineAffine coupling alternate'
# flows.append(piecaf_coup_alt), names.append(name)

# transformations = create_affinepiecewise_trans(num_trans, flow_forward)
# piecaf_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
# name = 'PiecewiseAffineAffine coupling random'
# flows.append(piecaf_coup_rand), names.append(name);

In [12]:
# #PiecewiseAffineAffine with AR

# transformations = create_affinepiecewise_trans(num_trans, flow_forward)
# piecaf_ar_id = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
# name = 'PiecewiseAffineAffine AR identity'
# flows.append(piecaf_ar_id), names.append(name)

# transformations = create_affinepiecewise_trans(num_trans, flow_forward)
# piecaf_ar_alt = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
# name = 'PiecewiseAffineAffine AR alternate'
# flows.append(piecaf_ar_alt), names.append(name)

# transformations = create_affinepiecewise_trans(num_trans, flow_forward)
# piecaf_ar_rand = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
# name = 'PiecewiseAffineAffine AR random'
# flows.append(piecaf_ar_rand), names.append(name);

In [13]:
# #ContinuousPiecewise with twoblock

# transformations = create_continuous_piecewise_trans(num_trans, flow_forward)
# conpiec_coup_iden = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
# name = 'ContinuousPiecewiseAffine coupling identity'
# flows.append(conpiec_coup_iden), names.append(name)

# transformations = create_continuous_piecewise_trans(num_trans, flow_forward)
# conpiec_coup_alt = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
# name = 'ContinuousPiecewiseAffine coupling alternate'
# flows.append(conpiec_coup_alt), names.append(name)

# transformations = create_continuous_piecewise_trans(num_trans, flow_forward)
# conpiec_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
# name = 'ContinuousPiecewiseAffine coupling random'
# flows.append(conpiec_coup_rand), names.append(name);

In [14]:
# #ContinuousPiecewise with AR

# transformations = create_continuous_piecewise_trans(num_trans, flow_forward)
# conpiec_ar_iden = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
# name = 'ContinuousPiecewiseAffine AR identity'
# flows.append(conpiec_ar_iden), names.append(name)

# transformations = create_continuous_piecewise_trans(num_trans, flow_forward)
# conpiec_ar_alt = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
# name = 'ContinuousPiecewiseAffine AR alternate'
# flows.append(conpiec_ar_alt), names.append(name)

# transformations = create_continuous_piecewise_trans(num_trans, flow_forward)
# conpiec_ar_rand = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
# name = 'ContinuousPiecewiseAffine AR random'
# flows.append(conpiec_ar_rand), names.append(name);

In [15]:
# #ContinuousPiecewiseAffine with twoblock

# transformations = create_affinecontinuous_trans(num_trans, flow_forward)
# affconpiec_coup_iden = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
# name = 'ContinuousPiecewiseAffineAffine coupling identity'
# flows.append(affconpiec_coup_iden), names.append(name)

# transformations = create_affinecontinuous_trans(num_trans, flow_forward)
# affconpiec_coup_alt = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
# name = 'ContinuousPiecewiseAffineAffine coupling alternate'
# flows.append(affconpiec_coup_alt), names.append(name)

# transformations = create_affinecontinuous_trans(num_trans, flow_forward)
# affconpiec_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
# name = 'ContinuousPiecewiseAffineAffine coupling random'
# flows.append(affconpiec_coup_rand), names.append(name);

In [16]:
# #ContinuousPiecewiseAffine with AR

# transformations = create_affinecontinuous_trans(num_trans, flow_forward)
# affconpiec_ar_iden = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
# name = 'ContinuousPiecewiseAffineAffine AR identity'
# flows.append(affconpiec_ar_iden), names.append(name)

# transformations = create_affinecontinuous_trans(num_trans, flow_forward)
# affconpiec_ar_alt = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[1], flow_forward)
# name = 'ContinuousPiecewiseAffineAffine AR alternate'
# flows.append(affconpiec_ar_alt), names.append(name)

# transformations = create_affinecontinuous_trans(num_trans, flow_forward)
# affconpiec_ar_rand = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
# name = 'ContinuousPiecewiseAffineAffine AR random'
# flows.append(affconpiec_ar_rand), names.append(name);

In [17]:
# #Alternating Piecewise Affine with twoblock

# transformations = create_alt_piecewise_affine_trans(num_trans, flow_forward)
# afpiec_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
# name = 'Alternating Affin_PiecewiseAffine coupling random'
# flows.append(afpiec_coup_rand), names.append(name);

In [18]:
# #Alternating Piecewise Affine with AR

# transformations = create_alt_piecewise_affine_trans(num_trans, flow_forward)
# afpiec_ar_rand = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
# name = 'Alternating Affin_PiecewiseAffine AR random'
# flows.append(afpiec_ar_rand), names.append(name);

In [19]:
# #Alternating with Linear layer and Affine

# transformations = create_alt_linear_affine_trans(num_trans, dim_input, flow_forward)
# linaff_coup_iden = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
# name = 'Alternating Linear_Affine coupling identity'
# flows.append(linaff_coup_iden), names.append(name)

# transformations = create_alt_linear_affine_trans(num_trans, dim_input, flow_forward)
# linaff_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
# name = 'Alternating Linear_Affine coupling random'
# flows.append(linaff_coup_rand), names.append(name)

# transformations = create_alt_linear_affine_trans(num_trans, dim_input, flow_forward)
# linaff_ar_iden = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[0], flow_forward)
# name = 'Alternating Linear_Affine AR identity'
# flows.append(linaff_ar_iden), names.append(name)

# transformations = create_alt_linear_affine_trans(num_trans, dim_input, flow_forward)
# linaff_ar_rand = create_flows_with_AR(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
# name = 'Alternating Linear_Affine AR random'
# flows.append(linaff_ar_rand), names.append(name);

In [20]:
# #Various Alternating Linear and Conditioner transformation

# transformations = create_alt_linear_piecewise_trans(num_trans, dim_input, flow_forward)
# linpiece_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
# name = 'Alternating Linear_PiecewiseAffine coupling random'
# flows.append(linpiece_coup_rand), names.append(name)

# transformations = create_alt_linear_continuous_trans(num_trans, dim_input, flow_forward)
# lincont_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
# name = 'Alternating Linear_ContinuousPiece coupling random'
# flows.append(lincont_coup_rand), names.append(name)

# transformations = create_alt_linear_affinepiecewise_trans(num_trans, dim_input, flow_forward)
# linaffpiec_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
# name = 'Alternating Linear_AffinePiece coupling random'
# flows.append(linaffpiec_coup_rand), names.append(name)

# transformations = create_alt_linear_affinecontinuous_trans(num_trans, dim_input, flow_forward)
# linaffcont_coup_rand = create_flows_with_twoblock(dim_input, dim_hidden, transformations, perm_types[2], flow_forward)
# name = 'Alternating Linear_AffineContinuous coupling random'
# flows.append(linaffcont_coup_rand), names.append(name);

In [21]:
losses = {n: [] for n in flows.keys()}
optimizers = {n: [] for n in flows.keys()}

for flowname in flows.keys():
    for j in range(num_exp):
        # Initialize flow
        flow = flows[flowname][j]
        update_device(device, flow, dataset)

        # Initialize optimizer
        optimizer = torch.optim.AdamW(flow.parameters(), lr=1e-4, weight_decay=1e-2)
        optimizers[flowname].append(optimizer)
        
        # Train and append losses
        losses[flowname].append(
            train_backward(
                flow, 
                dataset.get_training_data(),
                optimizer, 
                # Extra epochs for simple models
                epochs + 20 if flowname == 'Linear' else epochs + 50, 
                batch_size, 
                print_n=200, 
                save_checkpoint=True, 
                burn_in=-1
            )
        )

        # Move flow to CPU
        update_device(device_cpu, flow, dataset)

Finished training. Loss for last epoch Linear 4:     60.37162


In [22]:
best_flows = {n:[] for n in flows.keys()}
for flowname in flows.keys():
    for i in range(num_exp):
        best_flows[flowname].append(load_best_model(flows[flowname][i]))

In [23]:
log_scale = True
from_iter = 50 * batch_size

fig, ax = plt.subplots(2, len(flows), figsize=(12,6))
for j, flowname in enumerate(flows.keys()):
    for i in range(num_exp):
        ax[0,j].plot(losses[flowname][i], label=f'exp{i}', alpha=0.8)
        ax[0,j].set_title(f'{flowname}')
        if log_scale:
            ax[0, j].set_yscale('log')
        ax[0, j].legend()

log_scale = True
for j, flowname in enumerate(flows.keys()):
    for i in range(num_exp):
        x = np.arange(len(losses[flowname][i]))
        ax[1,j].plot(x[from_iter:], losses[flowname][i][from_iter:], label=f'exp{i}', alpha=0.8)
        if log_scale:
            ax[1, j].set_yscale('log')
        ax[1, j].legend()
    
plt.tight_layout()

In [24]:
flow_lh_results = {
    'train': {n:[] for n in flows}, 
    'test': {n:[] for n in flows}, 
    'sample': {n:[] for n in flows},
}
cur_res = 'train'

print('Results based on training data:' + '\n')

# Function for CI
def mean_confidence_interval(data, confidence=0.95):
    import scipy.stats
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h
 

train_data = dataset.get_training_data()
mean_target = torch.mean(dataset.evaluate(train_data.to(device))).detach().numpy()
for flow in best_flows:
    for i in range(num_exp):
        log_lik, mean = log_likelihood(train_data, flows[flow][i])
        flow_lh_results[cur_res][flow].append(mean[0])
        print("Mean loglikelihood for {}: {}".format(f'{flow} {i}', mean))
        print("Difference between target and {} mean loglikelihood: {}".format(str(flow), abs(mean-mean_target)))
        print()
    
    m, h = mean_confidence_interval(flow_lh_results[cur_res][flow])
    print('-'*75)
    print(f'{flow} -- Mean LogLH Bootstrap: {m:7.4f} ± {h:7.4f}')
    print('-'*75)
    print()
    

print("Mean loglikelihood with actual distribution: {}".format(mean_target))

Results based on training data:

Mean loglikelihood for Mean field 0: [-49.411015]
Difference between target and Mean field mean loglikelihood: [2.965229]

Mean loglikelihood for Mean field 1: [-49.7959]
Difference between target and Mean field mean loglikelihood: [3.350113]

Mean loglikelihood for Mean field 2: [-49.503036]
Difference between target and Mean field mean loglikelihood: [3.057251]

Mean loglikelihood for Mean field 3: [-49.67153]
Difference between target and Mean field mean loglikelihood: [3.2257462]

Mean loglikelihood for Mean field 4: [-49.514324]
Difference between target and Mean field mean loglikelihood: [3.0685387]

---------------------------------------------------------------------------
Mean field -- Mean LogLH Bootstrap: -49.5792 ±  0.1901
---------------------------------------------------------------------------

Mean loglikelihood for Linear 0: [-60.356667]
Difference between target and Linear mean loglikelihood: [13.910881]

Mean loglikelihood for Linear

In [25]:
print('Results based on test data' + '\n')
cur_res = 'test' 

test_data = dataset.get_test_data()
mean_target = torch.mean(dataset.evaluate(test_data)).detach().numpy()
for flow in best_flows:
    for i in range(num_exp):
        log_lik, mean = log_likelihood(test_data, flows[flow][i])
        flow_lh_results[cur_res][flow].append(mean[0])
        print("Mean loglikelihood for {}: {}".format(f'{flow} {i}', mean))
        print("Difference between target and {} mean loglikelihood: {}".format(str(flow), abs(mean-mean_target)))
        print()
    
    m, h = mean_confidence_interval(flow_lh_results[cur_res][flow])
    print('-'*75)
    print(f'{flow} -- Mean LogLH Bootstrap: {m:7.4f} ± {h:7.4f}')
    print('-'*75)
    print()
    

print("Mean loglikelihood with actual distribution: {}".format(mean_target))

# for flow in best_flows:
#     log_lik, mean = log_likelihood(test_data, flow)
#     print("Mean loglikelihood for {}: {}".format(str(flow), mean))
#     print("Difference between target and {} mean loglikelihood: {}".format(str(flow), abs(mean-mean_target)))
    

# print("Mean loglikelihood with actual distribution: {}".format(mean_target))

Results based on test data

Mean loglikelihood for Mean field 0: [-49.258945]
Difference between target and Mean field mean loglikelihood: [2.8089333]

Mean loglikelihood for Mean field 1: [-49.651436]
Difference between target and Mean field mean loglikelihood: [3.2014236]

Mean loglikelihood for Mean field 2: [-49.358437]
Difference between target and Mean field mean loglikelihood: [2.9084244]

Mean loglikelihood for Mean field 3: [-49.52597]
Difference between target and Mean field mean loglikelihood: [3.0759583]

Mean loglikelihood for Mean field 4: [-49.359097]
Difference between target and Mean field mean loglikelihood: [2.9090843]

---------------------------------------------------------------------------
Mean field -- Mean LogLH Bootstrap: -49.4308 ±  0.1940
---------------------------------------------------------------------------

Mean loglikelihood for Linear 0: [-59.78685]
Difference between target and Linear mean loglikelihood: [13.336838]

Mean loglikelihood for Linear 

In [26]:
print('Results based on new sample from each flow:' + '\n')
cur_res = 'sample'

for flow in best_flows:
    for i in range(num_exp):
        with torch.no_grad():
            sample, log_prob = flows[flow][i].sample(800)
            sample_last = sample[-1]

        mean_target = torch.mean(dataset.evaluate(sample_last)).detach().numpy()
        log_lik, mean = log_likelihood(sample_last, flows[flow][i])
        flow_lh_results[cur_res][flow].append(mean[0])
        print("Mean loglikelihood for {}: {}".format(f'{flow} {i}', mean))
        print("Difference between target and {} mean loglikelihood: {}".format(str(flow), abs(mean-mean_target)))
        print("Mean loglikelihood with actual distribution: {}".format(mean_target))   
        print()
    
    m, h = mean_confidence_interval(flow_lh_results[cur_res][flow])
    print('-'*75)
    print(f'{flow} -- Mean LogLH Bootstrap: {m:7.4f} ± {h:7.4f}')
    print('-'*75)
    print()


# for flow in best_flows:
#     with torch.no_grad():
#         sample, log_prob = flow.sample(800)
#         sample_last = sample[-1]

#     #sample_last.clip_(-7, 7)
#     mean_target = torch.mean(dataset.evaluate(sample_last)).detach().numpy()
#     log_lik, mean = log_likelihood(sample_last, flow)

#     print('Check for NANs: ', torch.isnan(sample_last).any().item())
#     print("Mean loglikelihood for {}: {}".format(str(flow), mean))
#     print("Difference between target and {} mean loglikelihood: {}".format(str(flow), abs(mean-mean_target)))    
#     print("Mean loglikelihood with actual distribution: {}".format(mean_target))   

Results based on new sample from each flow:

Mean loglikelihood for Mean field 0: [-50.220566]
Difference between target and Mean field mean loglikelihood: [2.0308533]
Mean loglikelihood with actual distribution: -52.25141906738281

Mean loglikelihood for Mean field 1: [-51.192734]
Difference between target and Mean field mean loglikelihood: [3.3065987]
Mean loglikelihood with actual distribution: -54.499332427978516

Mean loglikelihood for Mean field 2: [-50.553856]
Difference between target and Mean field mean loglikelihood: [2.2261848]
Mean loglikelihood with actual distribution: -52.7800407409668

Mean loglikelihood for Mean field 3: [-51.15684]
Difference between target and Mean field mean loglikelihood: [2.977436]
Mean loglikelihood with actual distribution: -54.13427734375

Mean loglikelihood for Mean field 4: [-50.51577]
Difference between target and Mean field mean loglikelihood: [2.3286438]
Mean loglikelihood with actual distribution: -52.84441375732422

---------------------